In [1]:
@noinline @fastmath function f_fastmath(x_prev, v, dt)
    x_prev + v*dt
end

"""
Solve x(0) = 0, x'(t) = x(t) + 1, 0 ≤ t ≤ 1.

Exact solution: x(t) = exp(t) - 1, x(1) = exp(1) - 1 = 1.718281828⋯.
"""
function solveode(f, n=10^6, x0=0.0, dt=1/n)
    x = x0
    for _ in 1:n
        x = f(x, x+1, dt)
    end
    x
end

function solveode_inline(f, n=10^6, x0=0.0, dt=1/n)
    x = x0
    for _ in 1:n
        x = @inline f(x, x+1, dt)
    end
    x
end

@show solveode(f_fastmath)
@show solveode_inline(f_fastmath)
@show exp(1) - 1;

solveode(f_fastmath) = 1.7182804693193663
solveode_inline(f_fastmath) = 1.7182804693193663
exp(1) - 1 = 1.718281828459045


In [2]:
using BenchmarkTools

@btime solveode(f_fastmath)
@btime solveode_inline(f_fastmath);

  1.750 ms (0 allocations: 0 bytes)
  54.400 μs (0 allocations: 0 bytes)


In [3]:
@code_native debuginfo=:none dump_module=false solveode(f_fastmath)

	.text
	push	rbp
	mov	rbp, rsp
	push	rsi
	push	rdi
	sub	rsp, 64
	vmovaps	xmmword ptr [rbp - 32], xmm7
	vmovaps	xmmword ptr [rbp - 48], xmm6
	mov	rax, qword ptr [r13 + 16]
	mov	rax, qword ptr [rax + 16]
	mov	rax, qword ptr [rax]
	vxorpd	xmm0, xmm0, xmm0
	mov	esi, 1000000
	movabs	rax, offset .rodata.cst8
	vmovsd	xmm7, qword ptr [rax]           # xmm7 = mem[0],zero
	movabs	rdi, offset f_fastmath
	movabs	rax, 2272748021016
	vmovsd	xmm6, qword ptr [rax]           # xmm6 = mem[0],zero
	nop
L80:
	vaddsd	xmm1, xmm0, xmm7
	vmovaps	xmm2, xmm6
	call	rdi
	dec	rsi
	jne	L80
	vmovaps	xmm6, xmmword ptr [rbp - 48]
	vmovaps	xmm7, xmmword ptr [rbp - 32]
	add	rsp, 64
	pop	rdi
	pop	rsi
	pop	rbp
	ret
	nop	word ptr cs:[rax + rax]


In [4]:
@code_native debuginfo=:none dump_module=false solveode_inline(f_fastmath)

	.text
	push	rbp
	mov	rbp, rsp
	mov	rax, qword ptr [r13 + 16]
	mov	rax, qword ptr [rax + 16]
	mov	rax, qword ptr [rax]
	vxorpd	xmm0, xmm0, xmm0
	mov	eax, 1000000
	movabs	rcx, offset .rodata.cst8
	vmovsd	xmm1, qword ptr [rcx]           # xmm1 = mem[0],zero
	movabs	rcx, 2272748022136
	vmovsd	xmm2, qword ptr [rcx]           # xmm2 = mem[0],zero
	nop	word ptr cs:[rax + rax]
L64:
	vfmadd213sd	xmm0, xmm1, xmm2        # xmm0 = (xmm1 * xmm0) + xmm2
	add	rax, -16
	jne	L64
	pop	rbp
	ret
	nop	dword ptr [rax]
